In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from calculations.common import (
    prepare_data,
    calculate_session_stats,
    calculate_page_stats,
)
from calculations.web_traffic import (
    aggregate_web_traffic_daily,
    calculate_social_media_correlation,
    aggregate_web_traffic_hourly,
    aggregate_web_traffic_weekday,
)
from calculations.device_type import calculate_desktop_mobile_stats,plot_device_pie_chart
from calculations.geolocation import aggregate_location, aggregate_location_per_day
from calculations.motion import calculate_motion_stats, filter_motion_events

In [ ]:
sns.set_theme(style="darkgrid")
colors = sns.color_palette('pastel')

start_date = '2022-02-17'
end_date = '2022-03-19'
df_raw = pd.read_csv('data/event_data_20220217-20220319.csv')
df_web = prepare_data(df_raw)
df_unique_users = df_web.groupby(by=['fingerprint']).first().reset_index()

df_facebook_reach = pd.read_csv('data/facebook_reach.csv', encoding = "utf-8")
df_instagram_reach = pd.read_csv('data/instagram_reach.csv', encoding = "utf-8")

df_facebook_pageviews = pd.read_csv('data/facebook_pageviews.csv', encoding = "utf-8")
df_instagram_pageviews = pd.read_csv('data/instagram_pageviews.csv', encoding = "utf-8")


In [ ]:
# Web traffic data
date_preproc = aggregate_web_traffic_daily(df_web)
hour_preproc = aggregate_web_traffic_hourly(df_web)
weekday_preproc = aggregate_web_traffic_weekday(date_preproc)
session_stats_overall = calculate_session_stats(df_web)

# Social media data
social_media_correlation = calculate_social_media_correlation(
    df_sessions_per_day=date_preproc['Sessioner'],
    df_facebook_reach=df_facebook_reach,
    df_facebook_pageviews=df_facebook_pageviews,
    df_instagram_reach=df_instagram_reach,
    df_instagram_pageviews=df_instagram_pageviews,
    date_days=list(date_preproc['Datum'])
)

# Page data
page_statistics = calculate_page_stats(df_web)

# Device data
desktop_events = df_web[
    df_web.where((df_web["device_type"] == "desktop")).device_type.notna()
]
mobile_events = df_web[
    df_web.where((df_web["device_type"] == "mobile")).device_type.notna()
]
desktop_mobile_stats = calculate_desktop_mobile_stats(df_web)

# Geolocation data
df_city_count = aggregate_location(df_unique_users, 'city', cut_off = 15)
df_city_count_daily = aggregate_location_per_day(df_unique_users, 'city', df_city_count.index)

df_country_count = aggregate_location(df_unique_users, 'country', cut_off = 7)
df_country_count_daily = aggregate_location_per_day(df_unique_users, 'country', df_country_count.index)

# Motion data
df_motions = filter_motion_events(df_web)

# Done 
print(f'\nData för hemsidan under perioden {start_date} till {end_date}')

# Resultat

Nedan följer statistik på trafik genom hemsidan (https://klimatriksdagen.se). Resultaten är baserad på tracking data som jag själv implementerat på hemsidan (varje gång en användare laddar en ny sida skickas information om vilket sida som laddades och liknande). Källkoden för den intresserade finns här: https://github.com/johanBlad/kr-22-web-analytics

Resultaten är uppdelade i 6 delar, där varje del har ett antal frågeställningar knytna till sig, som jag försökt besvara genom den datan jag har.


### [Website traffic](#Statistik-för-trafik-på-hemsidan-överlag)
- How is total website traffic fluctuating by day and by week? 
- How is website sessions fluctuating by day and by week?
- How many unique users visit the site by day and by week?
- What is the ratio of one-event-sessions vs multiple-event-sessions?
- What is the average number of page loads per session?
- How long is average time on page?
- How long is an average session (for sessions with more than one page load)?
- How does website traffic fluctuate during the day on average?
- How does website traffic fluctuate during the week on average?

### [Page statistics](#Statistik-per-Sida-på-Hemsidan) 
- What pages are the most common start of each session?
- What pages are the most common end of each session?
- What pages have the longest read time?
- What pages are most common visisted pages overall?
- What pages are the most common visisted pages over sessions? 
- What pages are the most common visisted pages over unique users?
- What ratio of sessions come in from Facebook
- Of visitors coming from Facebook, what pages do they land on?

### [Device type](#Statistik-på-hemsidan-mellan-Mobil-och-Webbläsare)
- What is the distribution between desktop and mobile for overall events?
- What is the distribution between desktop and mobile for sessions?
- What is the distribution between desktop and mobile for unique users?
- What is the ratio of one-event-sessions to multiple-event-sessions on desktop versus mobile?
- What is the average number of events per session on desktop versus mobile?
- How long is an average session (for sessions with more than one page load) on desktop versus mobile?
- Page statistics as done above, but split between desktop and mobile

### [Geolocation](#Geografisk-distribution-för-besökare-på-hemsidan)
- How does the distribution of countries look like for unique users?
- How does the distribution of cities look like for unique users?
- When splitting distributions of cities in n top cities + others, how is this developing over time?


### [Motion statistics](#Statistik-per-Motion-på-Hemsidan)
- What motions are most common visisted motions overall?
- What motions are the most common visisted motions over sessions? 
- What motions are the most common visisted motions over unique users? 


# Statistik för trafik på hemsidan överlag

## Sidvisningar, Sessioner & Unika Besökare i snitt över datum
**Sidvisningar (#):** Antalet totala sidvisningar på hemsidan över datum.

**Sessioner (#):** Antalet totala sessioner på hemsidan över datum.

*En session innefattar alla sidvisningar för en enskild besökare där det inte har gått längre än 30 minuter mellan två efterföljande sidvisningar. Om en besökare stannar på en sida under 31 minuter, för att sen går vidare till en annan sida på hemsidan, då startas en ny session. Sessioner som spänner mellan två dygn, alltså över midnatt, kapas vid midnatt och det blir istället två sessioner, en innan och en efter midnatt.*

**Unika Besökare (#):** Antalet unika besökare på hemsidan varje dag.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='Datum', y='value', hue='variable', data=pd.melt(
    date_preproc.drop(columns=['Veckodag']), ['Datum']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Antal')
plt.show()

## Sidvisningar & Sessioner i snitt över veckodag

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.barplot(x='Veckodag', y='value', hue='variable', data=pd.melt(weekday_preproc, ['Veckodag']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Snitt Antal')
plt.show()

## Sidvisningar & Sessioner i snitt över tid på dygnet

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.barplot(x='Timme', y='value', hue='variable', data=pd.melt(hour_preproc, ['Timme']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Snitt Antal')
plt.show()

## Antal Sidvisningar och Sidtid under Sessioner

In [ ]:
print(f"""
Andel av sessioner där besökaren endast besökte en enskild sida:\t {session_stats_overall['one_event_session_ratio']}%
Antal sidor som besökare i snitt besöker under en session:\t\t {session_stats_overall['avg_events_per_session']} stycken
Antal sidor som besökare i snitt besöker under sessioner med 
    fler än en sidvisning:\t\t\t\t\t\t {session_stats_overall['avg_events_per_multievent_sessions']} stycken

Snitt-tid som besökare spenderar
    på varje sida (median):\t\t\t {session_stats_overall['median_time_per_page']} sekunder
    på hemsidan under en session (median):\t {session_stats_overall['median_session_time']} sekunder
""")

## Hemsida Sessioner korrelation Facebook
Hur besökares sessioner på Hemsidan korrelerar (sammanfaller) med Räckvidd och Sidvisningar på Facebook

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='Datum', y='value', hue='variable', data=pd.melt(social_media_correlation['facebook'], ['Datum']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Intensitet')
plt.show()

## Hemsida Sessioner korrelation Instagram
Hur besökares sessioner på Hemsidan korrelerar (sammanfaller) med Räckvidd och Sidvisningar på Instagram

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='Datum', y='value', hue='variable', data=pd.melt(
    social_media_correlation['instagram'], ['Datum']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Intensitet')
plt.show()

# Statistik per Sida på Hemsidan

**Sidtid (median sekunder):** Medianen av den tid som besökare i snitt spenderar på denna sidan.

**Sidvisningar (#):** Antalet totala sidvisningar på denna sida.

**Sessioner (#):** Antalet totala sessioner där besökaren besökte denna sidan under sessionen.

**Unika Besökare (#):** Antalet unika besökare som har besökt denna sidan.

**Från Facebook (#):** Hur många besök på denna sidan som kommit från en länk på Facebook

**Första Sidan på Sessionen (#):** Antal gånger sidan har varit den första sidan som besökaren besökt som första sida under en session.

**Sista Sidan på Sessionen (#):** Antal gånger sidan har varit den sista sidan som besökaren besökt innan besökaren avslutade sessionen (lämnade hemsidan).

**Sannolikhet att Lämna Hemsidan: Beräknat (%):** Grovt uppskattad sannolikhet att det nästa användaren gör är att lämna hemsidan efter att ha besökt denna sida. Beräknat som `Sista Sidan på Sessionen / Sidvisningar` för varje sida.

In [ ]:
page_statistics

# Statistik på hemsidan mellan Mobil och Webbläsare

## Fördelning av besökare på Mobil och Webbläsare

In [ ]:
plot_device_pie_chart(desktop_mobile_stats)
plt.show()
desktop_mobile_stats


## Statistik per Sida på hemsidan för Webläsare 

In [ ]:
calculate_page_stats(desktop_events)

## Statistik per Sida på hemsidan för Mobil 

In [ ]:
calculate_page_stats(mobile_events)

# Geografisk distribution för besökare på hemsidan

## Besökare över stad

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.pie(df_city_count['fingerprint'], labels = df_city_count.index , colors = colors[0:len(df_city_count)], autopct='%.0f%%')
ax.set_title('Besökare över stad')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,8))
sns.lineplot(x='date_day', y='value', hue='city', data=pd.melt(df_city_count_daily, ['city', 'date_day']), ax=ax)
plt.xticks(rotation=45); plt.ylabel('Unika Besökare')
plt.show()

## Besökare över land


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
ax.pie(df_country_count['fingerprint'], labels = df_country_count.index , colors = colors[0:len(df_country_count.index)], autopct='%.0f%%')
ax.set_title('Besökare över land')
plt.show()

# Statistik per Motion på Hemsidan

**Sidtid (median sekunder):** Medianen av den tid som besökare i snitt spenderar på denna sidan.

**Sidvisningar (#):** Antalet totala sidvisningar för denna motion.

**Sessioner (#):** Antalet totala sessioner där besökaren besökte denna motion under sessionen.

**Unika Besökare (#):** Antalet unika besökare som har besökt denna motion.


In [ ]:
calculate_motion_stats(df_motions)[:20]